# **Detecção de Glaucoma em exames de Fundoscopia**

# **Organizar os arquivos**

In [ ]:
import os, os.path # Miscellaneous Operating System Interfaces

!pip install --upgrade pip # Update pip
!pip install scikit-image # add scikit no projeto, para o import
!pip install scikit-learn -U # add scikit no projeto, para o import
!pip install -U imbalanced-learn

!pip install git+https://github.com/ypeleg/nfnets-keras


# extração do dataset no Kaggle
!pip install kaggle
# Chave de uma conta do Kaggle (login no Kaggle, então: Profile>Account>API, criar API token, baixando kaggle.json)
os.environ['KAGGLE_USERNAME'] = 'jakijonas'
os.environ['KAGGLE_KEY'] = '62658aedc8305f0d14232ab83046ea1d'
# Extrair e Importar o Dataset do Reconhecimento de Doenças Oculares
!kaggle datasets list -s andrewmvd/ocular-disease-recognition-odir5k # list datasets matching a search term
# download files associated with a dataset
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k --force 
!unzip -q ocular-disease-recognition-odir5k.zip # Descompactar
# Remover arquivos desnecessários
!rm ocular-disease-recognition-odir5k.zip 
!rm -r -f sample_data/
!rm -r -f preprocessed_images/


In [ ]:
# Esse trecho é responsavel por organizar dentro dos arquivos corretos
# cada imagem. Dessa forma, as imagens classificadas com
# GLAUCOMA foram para o diretório database/1 e as imagens
# classificadas como NORMAIS foram para o diretorio database/0
import pandas as pd
import shutil
import random

if (not os.path.exists('database')):
  os.mkdir('database')
  os.mkdir('database/0')
  os.mkdir('database/1')

col_list = ['filename', 'filepath','N','G']
df = pd.read_csv('full_df.csv')
df.head()
# NO CSV, as seguintes colunas:
# coluna [04] = filename
# coluna [08] = NORMAL
# coluna [10] = GLAUCOMA
# coluna [16] = filepath

itertuples = df.itertuples(name=None) 

#print('tamanho da itertuples:\t{}'.format(len(df)))

normal_count = 0
glaucoma_count = 0


for i in itertuples:
  filename = i[4]
  source = 'ODIR-5K/'+ i[16][43:]
  #print('source: \'{}\''.format(source))
  N = i[8]
  G = i[10]

  if(N==1):
    normal_count += 1
    normal_dir = 'database/0/'+str(normal_count)+filename
    shutil.copy(source, normal_dir)
  elif(G==1):
    glaucoma_count += 1
    glaucoma_dir = 'database/1/'+str(glaucoma_count)+filename
    shutil.copy(source, glaucoma_dir)

print('arquivos classe 0 (Normal):\t{}'.format(normal_count))
print('arquivos classe 1 (Glaucoma):\t{}'.format(glaucoma_count))
print('qnt total de arquivos:\t{}'.format(normal_count + glaucoma_count))


arquivos classe 0 (Normal):	2101
arquivos classe 1 (Glaucoma):	397
qnt total de arquivos:	2498


In [ ]:
!ls /content/database/0 | wc -l
!ls /content/database/1 | wc -l

2101
397
